In [80]:
# creating paths to src and data folders in the repo
import sys
import pathlib
src_path = pathlib.Path().absolute().parent.parent / "src"
data_path = pathlib.Path().absolute().parent.parent / "data"

# add src path to sys.path so it is searched in import statements
sys.path.append(str(src_path))

# importing api wrappers for reddit data
import praw
from psaw import PushshiftAPI

# retrieve api credentials from .gitignore'd text file
secrets_path = src_path / 'api_credentials.txt'
secrets_txt = open(secrets_path, 'r')

my_id = secrets_txt.readline().split('=')[1].rstrip()
my_secret = secrets_txt.readline().split('=')[1].rstrip()

secrets_txt.close()

# create a praw and pushshitft instances
reddit = praw.Reddit(
     client_id=my_id,
     client_secret=my_secret,
     user_agent="test_script by u/Mizule_RL"
 )

s_api = PushshiftAPI(reddit)

import requests
import pandas as pd
import numpy as np
import datetime as dt
import time

In [96]:
start_epoch = int(dt.datetime(2020, 11, 16).timestamp())
data_size = 10000

collected_posts = []
while len(collected_posts) < data_size:
    print(f'polling pushshift for {data_size - len(collected_posts)} more posts before {start_epoch}')
    batch = list(s_api.search_submissions(before=start_epoch, subreddit='Art', limit=1000))
    for post in batch:
        if len(post.title.split(',')) == 4:
            collected_posts.append(post)
    start_epoch = int(batch[-1].created_utc)
    
len(collected_posts)

polling pushshift for 10000 more posts before 1605506400
polling pushshift for 9942 more posts before 1605494190
polling pushshift for 9868 more posts before 1605483669
polling pushshift for 9798 more posts before 1605474400
polling pushshift for 9736 more posts before 1605467638
polling pushshift for 9675 more posts before 1605458989
polling pushshift for 9607 more posts before 1605450977
polling pushshift for 9532 more posts before 1605436347
polling pushshift for 9470 more posts before 1605418521
polling pushshift for 9400 more posts before 1605404941
polling pushshift for 9339 more posts before 1605396258
polling pushshift for 9271 more posts before 1605386693
polling pushshift for 9206 more posts before 1605378135
polling pushshift for 9136 more posts before 1605369618
polling pushshift for 9073 more posts before 1605359014
polling pushshift for 9014 more posts before 1605345430
polling pushshift for 8958 more posts before 1605331360
polling pushshift for 8913 more posts before 16

polling pushshift for 712 more posts before 1603825509
polling pushshift for 648 more posts before 1603817430
polling pushshift for 585 more posts before 1603809450
polling pushshift for 515 more posts before 1603798745
polling pushshift for 452 more posts before 1603781240
polling pushshift for 396 more posts before 1603769111
polling pushshift for 339 more posts before 1603760427
polling pushshift for 291 more posts before 1603750853
polling pushshift for 226 more posts before 1603743085
polling pushshift for 165 more posts before 1603733557
polling pushshift for 110 more posts before 1603725542
polling pushshift for 49 more posts before 1603715994
polling pushshift for 3 more posts before 1603702773


10051

In [113]:
post_dicts = [{'title': post.title, 'medium': (post.title.split(',')[2].lower().strip()), 'url': post.url, 'id': post.id, 'unix_time': int(post.created_utc)} for post in collected_posts]

In [114]:
df = pd.DataFrame(post_dicts)

In [116]:
df.medium.value_counts()

digital                                      2936
acrylic                                       398
me                                            309
digital art                                   301
watercolor                                    234
                                             ... 
sketch photoshop collage                        1
mixed media water color and acrylic paint       1
chalk on sidewalk                               1
graphite and marker                             1
ballpointopen on paper                          1
Name: medium, Length: 1804, dtype: int64

In [115]:
list(df.medium.value_counts().index)

['digital',
 'acrylic',
 'me',
 'digital art',
 'watercolor',
 'oil on canvas',
 'pencil',
 'photography',
 'acrylic on canvas',
 'ink',
 'oil',
 'digital painting',
 'procreate',
 'mixed media',
 'photoshop',
 'oils',
 'acrylics',
 'graphite',
 'pen',
 '2020',
 'watercolour',
 'gouache',
 'charcoal',
 'colored pencils',
 'digital drawing',
 'ink on paper',
 'colored pencil',
 'markers',
 'digital collage',
 'pen and ink',
 'pencils',
 'acrylic paint',
 'digital illustration',
 'acrylics on canvas',
 'oil painting',
 'paper',
 'drawing',
 'digital 3d',
 'pastel',
 'oil on linen',
 'stained glass',
 'marker',
 'pencil on paper',
 'digital 2d',
 'ballpoint pen',
 'watercolors',
 'fineliner',
 'oil pastel',
 'pen on paper',
 'illustrator',
 'watercolor and ink',
 'watercolor on paper',
 '3d',
 'oil on panel',
 'soft pastels',
 'oil pastels',
 'pixel art',
 'traditional',
 'medium',
 'polymer clay',
 'acrylic on paper',
 'pencil and paper',
 'color pencil',
 'graphite on paper',
 'graphite

In [130]:
df[df.medium == 'gouache on arches block']

,title,medium,url,id,unix_time
1042,"Peace, me, gouache on Arches block, 2019",gouache on arches block,https://i.redd.it/s6k5z15o45z51.jpg,jtwhr8,1605331309


In [55]:
int(dt.datetime(2018, 12, 31).timestamp())

1546236000

In [73]:
y = 'abcd'
z = y.split(',')
z

['abcd']

In [74]:
for x in testing_psaw:
    print(x.link_flair_text)

Rule 1 - Title format
Artwork
Artwork
Rule 1
Artwork
Discussion
Artwork
Artwork
Artwork
Artwork


In [58]:
testing_psaw[-1].created_utc

1546233444.0

In [12]:
for submission in reddit.subreddit("Art").hot(limit=10):
    print(type(submission.title))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [60]:
for submission in reddit.subreddit("Art").hot(limit=10):
    response = requests.get(submission.url)

    file = open(submission.id, "wb")
    file.write(response.content)
    file.close()